In [1]:
# This program generates dates using the MBBG stochastic model. 
# 15 de enero de 2024
# Beatriz Bonilla-Capilla

In [2]:
import random
import math
import numpy as np
def generate():
    while True:
        V1 =(np.random.rand()*2) - 1.0
        V2 =(np.random.rand()*2) - 1.0
        W = V1 * V1 + V2 * V2
        if (W < 1.0):
            LW = math.log(W) / W
            LW = math.sqrt(-2.0 * LW)
            G1 = V1 * LW
            G2 = V2 * LW
    
            return G1, G2

# Ejemplo de uso:
G1, G2 = generate()
print("G1:", G1)
print("G2:", G2)

G1: 0.1289971085925929
G2: -1.0439397581071364


In [3]:
def HW(dta):
    #subroutina que usa generate para obtener W(t)
    g1,g2=generate()
    w1=g1
    w2=g2
    dw=(g2-g1)*np.sqrt(dta)
    return dw

In [4]:
def estocasticaxM(nobs,a,s,b,T,n,x0,t):
    n1=len(t)
    x2=np.zeros(n1)

    for k in range(0,n1):
        sumaE=0
        for i in range(k-1):
            #print('tk','ti',t[i+1]-t[i])
            dta=t[i+1]-t[i]
            sumaE=sumaE+np.exp(-b*(t[k]-t[i]))*HW(dta)
        r=(a-(s**2)/2.)/b
        #print(sumaE)
        x2[k]=nobs*np.exp(r+(np.log(x0/nobs)-r)*np.exp(-b*t[k])+ s*sumaE) #la parte estocástica
    return x2

In [5]:
def EstimacionA(nobs,s,b,x0,xT,T,t,x2):# b,x0,T,tiempo t, x(t)
    sumLog=0
    n1=len(t)
    #print('nt a',nt)
    for i in range(0,n1-1):
        #print(i, i+1)
        dl2=np.log(x2[i+1])+np.log(x2[i]) # los datos estan en x2
        dt=t[i+1]-t[i]
        #print(dl*dt)
        sumLog=sumLog+dl2*dt
        #print(sumLog)
    Esti_a=s**2/2.+(1/T)*(np.log(x2[-1])-np.log(x0)+(b/2.)*sumLog)
    Esti_alpha=Esti_a-b*np.log(nobs)
    return Esti_alpha 

In [6]:
def EstimacionB(s,x0,xT,T,t,x2):# b,x0,T,tiempo t, x(t)
    n1=len(t)
    #print('b nt',nt)
    # Primera Sumatoria en el numerador es igual a la primera sumatoria en el denominador
    sumLog1=0
    
    for i in range(0,n1-1):
        #print(i, i+1)
        dl2=np.log(x2[i+1])+np.log(x2[i]) # los datos estan en x2
        dt=t[i+1]-t[i]
        #print(dl*dt)
        sumLog1=sumLog1+dl2*dt
    
   
    ## segunda Sumatoria en el denominador
    sumLog3=0
    
    for i in range(0,n1-1):
        #print(i, i+1)
        dl2=(np.log(x2[i+1]))**2+(np.log(x2[i]))**2 # los datos estan en x2
        dt=t[i+1]-t[i]
        #print(dl*dt)
        sumLog3=sumLog3+dl2*dt
    #
        #print(sumLog)
    Esti_b0=(T)*((np.log(x2[-1]))**2-(np.log(x0))**2-(s**2)*T)
    Esti_b1=Esti_b0-(np.log(x2[-1])-np.log(x0))*sumLog1
    Esti_b2=(1/2)*(sumLog1)**2-T*(sumLog3)
    Esti_b=Esti_b1/Esti_b2
    return Esti_b

In [7]:
import time

In [8]:
def DatosGeneradosEstimacion_ABS(nobs,a,s,b,x0,T,n,np2): #np2 es el número de corridas
    t=np.linspace(0,T,n)
    n1=len(t)
    
    TN=t[n-1]
    #print(nt1)
    #print(TN)
    x2_varios=np.zeros([np2,n1])
    inicio=time.time() #--- medir el tiempo inicio
    for l in range(np2):
        x2_estocastica=estocasticaxM(nobs,a,s,b,T,n,x0,t)
    #print(l,len(parte_estocastica),parte_estocastica)
     #la parte estocástica
    #print(x2_var)
        x2_varios[l,:]=x2_estocastica
    #len(x2_varios)
    #print(x2_varios)
    fin=time.time() #--- medir el tiempo final
    #print('para T=',T,' n=',n,'ciclos=', np2)
    #print('tiempo(segundos) de generar los datos aleátorios',fin-inicio,'seg.')
    a_est=[]
    b_est=[]

    inicio=time.time() #--- medir el tiempo inicio

    for i in range(np2):
        x2_temp=x2_varios[i,:]
        xT=x2_temp[-1]
        #print('xT=',xT)
        #print('xT=',xT)
        #print(x2_temp[nt-1])
        Esti_a=EstimacionA(nobs,s,b,x0,xT,T,t,x2_temp)
        Esti_b=EstimacionB(s,x0,xT,T,t,x2_temp)

        #print(Esti_a*3)
        a_est=np.append(a_est,Esti_a)
        b_est=np.append(b_est,Esti_b)

    #Resultados finales
    a_techo=np.mean(a_est)
    b_techo=np.mean(b_est)
      
    a_est2=[]
    for i in range(np2):
        x2_temp=x2_varios[i,:]
        Esti_a2=EstimacionA(nobs,s,b_techo,x0,xT,T,t,x2_temp)
        
        #print(Esti_a*3)
        a_est2=np.append(a_est2,Esti_a2)
       
    #Resultados finales
    
    a_techo2=np.mean(a_est2)
    a_techo2_m=np.median(a_est2)
    a_techo2_q=np.quantile(a_est2,0.75)
    a_techo2_s=np.std(a_est2)

    
    #print('media=',a_techo2)
    #print('mediana=',a_techo2_m)
    #print('quartile=',a_techo2_q)
    #print('std=',a_techo2_s)
    
    
    fin=time.time() #--- medir el tiempo final
    tardanza=fin-inicio
    #print('----------------')
    #print('para T=',T,' n=',n,'ciclos=', np2)
    #print('tiempo(segundos) de estimar a techo y calcular su media=',fin-inicio,'seg.')
    
    #x3=estocasticax(a_techo,s,b,T,n,x0,t)
    #plt.plot(t,x,'r-')
    #plt.plot(t,x3,'o')
    #print('resultados')
    #print('a,b,s,n,T,a_techo,b_techo,a_techo2,dif_a,tardanza')
    dif_a=a_techo-a_techo2
    return nobs,a,b,s,n,T,a_techo,b_techo,a_techo2,dif_a,np2,a_techo2_m,a_techo2_q,a_techo2_s,tardanza
    

In [9]:
import pandas as pd

# Main Program

In [12]:
nobs=0.005
a=0.8
x0=0.5
T=30
n=128
np2=500 #ciclos

s1=[0,0.025,0.1,0.5,1.,1.1,1.2,1.3,1.4,1.5,2.]

In [15]:
s1=[0,0.025,0.1,0.2,0.4,0.5,0.6,0.7,0.8,1.0,1.2]

b1=[0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.10,
    0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,
    0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,
    0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,0.39,0.40,
    0.41,0.42,0.43,0.44,0.45,0.46,0.47,0.48,0.49,0.50,
    0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.58,0.59,0.60,
    0.61,0.62,0.63,0.64,0.65,0.66,0.67,0.68,0.69,0.70]

In [16]:
b1=np.linspace(0.001,0.8,100)
print(b1)


[0.001      0.00907071 0.01714141 0.02521212 0.03328283 0.04135354
 0.04942424 0.05749495 0.06556566 0.07363636 0.08170707 0.08977778
 0.09784848 0.10591919 0.1139899  0.12206061 0.13013131 0.13820202
 0.14627273 0.15434343 0.16241414 0.17048485 0.17855556 0.18662626
 0.19469697 0.20276768 0.21083838 0.21890909 0.2269798  0.23505051
 0.24312121 0.25119192 0.25926263 0.26733333 0.27540404 0.28347475
 0.29154545 0.29961616 0.30768687 0.31575758 0.32382828 0.33189899
 0.3399697  0.3480404  0.35611111 0.36418182 0.37225253 0.38032323
 0.38839394 0.39646465 0.40453535 0.41260606 0.42067677 0.42874747
 0.43681818 0.44488889 0.4529596  0.4610303  0.46910101 0.47717172
 0.48524242 0.49331313 0.50138384 0.50945455 0.51752525 0.52559596
 0.53366667 0.54173737 0.54980808 0.55787879 0.56594949 0.5740202
 0.58209091 0.59016162 0.59823232 0.60630303 0.61437374 0.62244444
 0.63051515 0.63858586 0.64665657 0.65472727 0.66279798 0.67086869
 0.67893939 0.6870101  0.69508081 0.70315152 0.71122222 0.71929

In [17]:
obs=0.005
a=0.8
x0=0.5
T=30
n=128
np2=500 #ciclos

datosfinales08=np.zeros([11,100,15])
for i in range(11): #ciclo para s, variando s desde 0, hasta 1
    s=s1[i]
    print(s)
    for k in range(100): # ciclo para b, Variando a desde 0.1 a 0.85 de 0.05
        #print('i=',i,'k=',k,'s=',s,'b=',b)
        b=b1[k]
        datos_temporal=DatosGeneradosEstimacion_ABS(nobs,a,s,b,x0,T,n,np2)
        datosfinales08[i,k,:]=datos_temporal # i es para s, # k es para b, # tiempo
        #print(b)
    

np.save('datosF_MBBG_nobs_005_28junio2024.npy',np.array(datosfinales08))#mas puntos en la zona adecuada


0
0.025
0.1
0.2
0.4
0.5
0.6
0.7
0.8
1.0
1.2
ya acabe


In [18]:
nobs=0.01
a=0.8
x0=0.5
T=30
n=128
np2=500 #ciclos

datosfinales08=np.zeros([11,100,15])
for i in range(11): #ciclo para s, variando s desde 0, hasta 1
    s=s1[i]
    print(s)
    for k in range(100): # ciclo para b, Variando a desde 0.1 a 0.85 de 0.05
        #print('i=',i,'k=',k,'s=',s,'b=',b)
        b=b1[k]
        datos_temporal=DatosGeneradosEstimacion_ABS(nobs,a,s,b,x0,T,n,np2)
        datosfinales08[i,k,:]=datos_temporal # i es para s, # k es para b, # tiempo
        #print(b)
    
np.save('datosF_MBBG_nobs_01_28junio2024.npy',np.array(datosfinales08))
print('ya acabe')

0
0.025
0.1
0.2
0.4
0.5
0.6
0.7
0.8
1.0
1.2
ya acabe


In [19]:
nobs=0.05
a=0.8
x0=0.5
T=30
n=128
np2=500 #ciclos


datosfinales08=np.zeros([11,100,15])
for i in range(11): #ciclo para s, variando s desde 0, hasta 1
    s=s1[i]
    print(s)
    for k in range(100): # ciclo para b, Variando a desde 0.1 a 0.85 de 0.05
        #print('i=',i,'k=',k,'s=',s,'b=',b)
        b=b1[k]
        datos_temporal=DatosGeneradosEstimacion_ABS(nobs,a,s,b,x0,T,n,np2)
        datosfinales08[i,k,:]=datos_temporal # i es para s, # k es para b, # tiempo
        #print(b)
    
np.save('datosF_MBBG_nobs_05_28junio2024.npy',np.array(datosfinales08))
print('ya acabe')

0
0.025
0.1
0.2
0.4
0.5
0.6
0.7
0.8
1.0
1.2
ya acabe


In [20]:
nobs=0.1
a=0.8
x0=0.5
T=30
n=128
np2=500 #ciclos

datosfinales08=np.zeros([11,100,15])
for i in range(11): #ciclo para s, variando s desde 0, hasta 1
    s=s1[i]
    print(s)
    for k in range(100): # ciclo para b, Variando a desde 0.1 a 0.85 de 0.05
        #print('i=',i,'k=',k,'s=',s,'b=',b)
        b=b1[k]
        datos_temporal=DatosGeneradosEstimacion_ABS(nobs,a,s,b,x0,T,n,np2)
        datosfinales08[i,k,:]=datos_temporal # i es para s, # k es para b, # tiempo
        #print(b)
    

np.save('datosF_MBBG_nobs_1_28junio2024.npy',np.array(datosfinales08))
print('ya acabe')

0
0.025
0.1
0.2
0.4
0.5
0.6
0.7
0.8
1.0
1.2
ya acabe


In [21]:
nobs=0.5
a=0.8
x0=0.5
T=30
n=128
np2=500 #ciclos

datosfinales08=np.zeros([11,100,15])
for i in range(11): #ciclo para s, variando s desde 0, hasta 1
    s=s1[i]
    print(s)
    for k in range(100): # ciclo para b, Variando a desde 0.1 a 0.85 de 0.05
        #print('i=',i,'k=',k,'s=',s,'b=',b)
        b=b1[k]
        datos_temporal=DatosGeneradosEstimacion_ABS(nobs,a,s,b,x0,T,n,np2)
        datosfinales08[i,k,:]=datos_temporal # i es para s, # k es para b, # tiempo
        #print(b)
    

np.save('datosF_MBBG_nobs_5_28junio2024.npy',np.array(datosfinales08))
print('ya acabe')

0
0.025
0.1
0.2
0.4
0.5
0.6
0.7
0.8
1.0
1.2
ya acabe
